In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

## Data Extraction

In [2]:
loans_df = pd.read_csv('../loan.csv',encoding='utf-8')

/Users/aarora7/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Cleaning

In [3]:
#Check if there any columns, with all rows null. Remove such columns
loans_df = loans_df.dropna(how='all',axis=1)

In [4]:
#Check if there are columns with just 1, or 2(null included) unique value, which makes them useless
for col in loans_df.columns:
    unique_vals = loans_df[col].unique()
    num_uniques = len(unique_vals)
    if num_uniques <= 2:
        print (col, unique_vals)

term [' 36 months' ' 60 months']
pymnt_plan ['n']
initial_list_status ['f']
collections_12_mths_ex_med [ 0. nan]
policy_code [1]
application_type ['INDIVIDUAL']
acc_now_delinq [0]
chargeoff_within_12_mths [ 0. nan]
delinq_amnt [0]
tax_liens [ 0. nan]


#### Removing the following columns, since they don't contribute to any variance
pymnt_plan ['n']
initial_list_status ['f']
collections_12_mths_ex_med [ 0. nan]
policy_code [1]
application_type ['INDIVIDUAL']
acc_now_delinq [0]
chargeoff_within_12_mths [ 0. nan]
delinq_amnt [0]
tax_liens [ 0. nan]

In [5]:
loans_df = loans_df.drop(['pymnt_plan','initial_list_status','policy_code','delinq_amnt','acc_now_delinq','application_type','collections_12_mths_ex_med','chargeoff_within_12_mths','tax_liens'], axis=1)

#### Remove columns that are extremely correlated from each other

In [6]:
threshold = 0.98
# Create correlation matrix
corr_matrix = loans_df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

In [7]:
print(to_drop)
print(upper)

['member_id', 'funded_amnt', 'out_prncp_inv']
                         id  member_id  loan_amnt  funded_amnt  \
id                      NaN    0.99365   0.141919     0.152286   
member_id               NaN        NaN   0.140710     0.150322   
loan_amnt               NaN        NaN        NaN     0.981578   
funded_amnt             NaN        NaN        NaN          NaN   
funded_amnt_inv         NaN        NaN        NaN          NaN   
installment             NaN        NaN        NaN          NaN   
annual_inc              NaN        NaN        NaN          NaN   
dti                     NaN        NaN        NaN          NaN   
delinq_2yrs             NaN        NaN        NaN          NaN   
inq_last_6mths          NaN        NaN        NaN          NaN   
mths_since_last_delinq  NaN        NaN        NaN          NaN   
mths_since_last_record  NaN        NaN        NaN          NaN   
open_acc                NaN        NaN        NaN          NaN   
pub_rec                 NaN   

In [8]:
##Redundant columns 'member_id', 'funded_amnt', 'out_prncp_inv' since they are very
##highly correlated with 'id', loan_amnt, out_prncp
loans_df = loans_df.drop(to_drop, axis=1)

In [9]:
#Final loan_df shape
print(loans_df.shape)

(39717, 45)


In [10]:
loans_df.columns

Index(['id', 'loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
       'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'url',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'pub_rec_bankruptcies'],
      dtype='object')

### Bivariate Analysis

In [11]:
##Let's check if zip code affects loan default

In [39]:
zipcode_pivot_df =loans_df.pivot_table(index='zip_code',columns='loan_status', values='id',aggfunc='count',fill_value=0)

In [40]:
zipcode_pivot_df['default_ratio_loanscompleted%'] = (zipcode_pivot_df['Charged Off']/(zipcode_pivot_df['Fully Paid']+ zipcode_pivot_df['Charged Off']))*100

In [41]:
zipcode_pivot_df.sort_values(by='default_ratio_loanscompleted%', inplace=True, ascending=False)

In [44]:
zipcode_pivot_df['total_loans_completed'] =zipcode_pivot_df['Fully Paid']+ zipcode_pivot_df['Charged Off']

In [45]:
zipcode_pivot_df.loc[(zipcode_pivot_df['default_ratio_loanscompleted%'] > 70) & (zipcode_pivot_df.total_loans_completed>5)]

loan_status,Charged Off,Current,Fully Paid,default_ratio_loanscompleted%,total_loans_completed
zip_code,,,,,
561xx,5,0,2,71.428571,7
746xx,5,0,2,71.428571,7
